## Initiate the vissim instance

In [1]:
# COM-Server
import win32com.client as com
import igraph
import matplotlib.pyplot as plt
import qgrid
from VISSIM_helpers import VissimRoadNet
from os.path import abspath
import pandas as pd

Add autocompletion for VISSIM COM Object

In [2]:
from IPython.utils.generics import complete_object

@complete_object.register(com.DispatchBaseClass)
def complete_dispatch_base_class(obj, prev_completions):
    try:
        ole_props = set(obj._prop_map_get_).union(set(obj._prop_map_put_))
        return list(ole_props) + prev_completions
    except AttributeError:
        pass

Start Vissim and load constants

In [3]:
Vissim = com.gencache.EnsureDispatch("Vissim.Vissim")
from win32com.client import constants as c

Load the test network

In [4]:
FileName = abspath(r"..\SO sim files\Vol100per.inpx")
Vissim.LoadNet(FileName)

Read dynamic assignment network

In [5]:
vis_net = Vissim.Net
vis_net.Paths.ReadDynAssignPathFile()

In [6]:
network_graph = VissimRoadNet(vis_net)

In [7]:
ref_edge_list = pd.read_pickle("edges_attr.pkl.gz")
assert (network_graph.visedges['ToNode'] == ref_edge_list['ToNode']).all()

In [8]:
Vissim.Graphics.CurrentNetworkWindow.SetAttValue("QuickMode", 1)

In [9]:
def time_paths():
    paths = {}
    while True:
        Vissim.Simulation.RunSingleStep()
        new_vehs = vis_net.Vehicles.GetDeparted()
        vehs = new_vehs.GetMultipleAttributes(['No', 'OrigParkLot', 'DestParkLot', 'Path'])
        for veh in vehs:
            paths[int(veh[0])] = network_graph.parking_lot_route(int(veh[1]), int(veh[2]))
        if Vissim.Simulation.SimulationSecond > 300:
            break
    return paths

%prun paths = time_paths()

In [86]:
Vissim.Exit()